## Scrapping et nettoyage des données

### Scrapping

In [ ]:
# installation des packages nécessaires

# !pip install beautifulsoup4
# !pip install requests

In [103]:
# importation des librairies

from requests import get 
from bs4 import BeautifulSoup
import pandas as pd

In [104]:
# importation de l'url

url_1 = "https://www.expat-dakar.com/refrigerateurs-congelateurs?page=2"

In [105]:
# récupération du contenu de la page

source = get(url_1)
source.text # voir le contenu de la page récupérée
# source

'<!DOCTYPE html>\n<html\n    lang="fr"\n    class=""\n    prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"\n><head><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><meta name="imagemode" content="force"/><link rel="dns-prefetch" href="https://i.roamcdn.net"><link rel="preconnect" href="https://i.roamcdn.net"><link rel="dns-prefetch" href="https://adservice.google.com"><link rel="preconnect" href="https://adservice.google.com"><link rel="dns-prefetch" href="https://cdn.cookielaw.org"><link rel="preconnect" href="https://cdn.cookielaw.org"><link rel="dns-prefetch" href="https://cdn.onesignal.com"><link rel="preconnect" href="https://cdn.onesignal.com"><link rel="dns-prefetch" href="https://connect.facebook.net"><link rel="preconnect" href="https://connect.facebook.net"><link rel="dns-prefetch" href="https://creativecdn.com"><link rel="preconnect" href="https://creativecdn.com"><link rel="dns-prefetch" href="https://googleads.g.doubleclick.net">

In [106]:
# vérification de la récupération de contenu
source.status_code

200

In [107]:
# stockage de la page dans beautifulsoup

soup = BeautifulSoup(source.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="" lang="fr" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"><head><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta content="force" name="imagemode"><link href="https://i.roamcdn.net" rel="dns-prefetch"/><link href="https://i.roamcdn.net" rel="preconnect"/><link href="https://adservice.google.com" rel="dns-prefetch"/><link href="https://adservice.google.com" rel="preconnect"/><link href="https://cdn.cookielaw.org" rel="dns-prefetch"/><link href="https://cdn.cookielaw.org" rel="preconnect"/><link href="https://cdn.onesignal.com" rel="dns-prefetch"/><link href="https://cdn.onesignal.com" rel="preconnect"/><link href="https://connect.facebook.net" rel="dns-prefetch"/><link href="https://connect.facebook.net" rel="preconnect"/><link href="https://creativecdn.com" rel="dns-prefetch"/><link href="https://creativecdn.com" rel="preconnect"/><link href="https://googleads.g.doubleclick.net" rel="dns-prefetch"/><link

In [108]:
# récupérer les conteneurs qui contiennent les informations : ce sont les blocs refrigerateurs/congelateurs

conteneurs = soup.find_all('div', {'class': 'listings-cards__list-item'})
# conteneurs = soup.find_all('div', class_ = 'listings-cards__list-item ')

In [109]:
# nombre de conteneurs
len(conteneurs)

10

1. Récupérer les infos sur chaque conteneur (détails, état, adresse, prix, image_lien)

In [112]:
## récupérer les informations sur chaque conteneur 
conteneur_unique = conteneurs[0]

In [113]:
# récupération "details"

details = conteneur_unique.find('div', {'class': 'listing-card__header__title'}).text.strip()
details


'FRIGO COMBINÉ ASTECH FC 180S 3 tiroirs'

In [114]:
# récupération "etat"
etat = conteneur_unique.find('span', {'class': 'listing-card__header__tags__item listing-card__header__tags__item--condition listing-card__header__tags__item--condition_new'}).text
etat

'Neuf'

In [115]:
# récupération "adresse"
adresse = conteneur_unique.find('div', {'class': 'listing-card__header__location'}).text.strip().replace(',\n', '').strip()
adresse

'Plateau                                                                                Dakar'

In [128]:
# récupération "prix"
prix = conteneur_unique.find('span', {'class': 'listing-card__price__value 1'}).text.strip().replace('\u202f', '').replace(' F Cfa', '')
prix

'210000'

In [129]:
# récupération "image _lien"
image_lien = soup.find('img', {'class': 'listing-card__image__resource vh-img'})['src']
image_lien

'https://i.roamcdn.net/hz/ed/listing-thumb-224w/c25ca2b46a90e4084b4ee9bc5280a66c/-/horizon-files-prod/ed/picture/q448zmnr/c88226e22416a0e5d39879dc131cb694fa7dc44b.jpg'

In [130]:
# Afficher toutes les infos récupérées
print("Détails : " + details,
      "\nEtat : " + str(etat),
      "\nAdresse : " + adresse,
      "\nPrix : " + prix,
      "\nImage_lien : " + image_lien)

Détails : Congelateur astech vertical 6 tiroirs plaque aluminium 
Etat : Neuf 
Adresse : Nord foire                                                                                Dakar 
Prix : 210000 
Image_lien : https://i.roamcdn.net/hz/ed/listing-thumb-224w/c25ca2b46a90e4084b4ee9bc5280a66c/-/horizon-files-prod/ed/picture/q448zmnr/c88226e22416a0e5d39879dc131cb694fa7dc44b.jpg


2. Généraliser sur tous les conteneurs

In [131]:
# Initialiser une liste pour stocker les données

base = []

for conteneur_unique in conteneurs:
    try:
        conteneurs = soup.find_all('div', {'class': 'listings-cards__list-item'})
        details = conteneur_unique.find('div', {'class': 'listing-card__header__title'}).text.strip()
        etat = conteneur_unique.find('span', {'class': 'listing-card__header__tags__item listing-card__header__tags__item--condition listing-card__header__tags__item--condition_new'}).text
        adresse = conteneur_unique.find('div', {'class':'listing-card__header__location'}).text.strip().replace(',\n','').strip()
        prix = soup.find('span', {'class': 'listing-card__price__value 1'}).text.strip().replace('\u202f', '').replace(' F Cfa', '')
        image_lien = soup.find('img', {'class': 'listing-card__image__resource vh-img'})['src']

        dict =  {'details':details, 'etat':etat, 'adresse':adresse, 'prix':prix, 'image_lien':image_lien}
        base.append(dict)
    except:
        pass

base_ = pd.DataFrame(base)

In [132]:
# affichage de la base
base_.head()

,details,etat,adresse,prix,image_lien
0,FRIGO COMBINÉ ASTECH FC 180S 3 tiroirs,Neuf,Plateau ...,210000,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
1,Kit Frigo portatif Solaire voiture,Neuf,Keur Massar ...,210000,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
2,Réfrigérateur bar 2 porte smart technologie,Neuf,Plateau ...,210000,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
3,Réfrigérateur bar 1 porte smart technologie,Neuf,Plateau ...,210000,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
4,Frigo vitrine ROCH RSF-300 300L,Neuf,VDN ...,210000,https://i.roamcdn.net/hz/ed/listing-thumb-224w...


In [133]:
# vérification de la dimension
base_.shape

(9, 5)

In [134]:
# scraper sur toutes pages

base_donnee = pd.DataFrame()

for page in range(1,223):
    url_1 = f'https://www.expat-dakar.com/refrigerateurs-congelateurs?page={page}'
    source = get(url_1)
    soup = BeautifulSoup(source.text, 'html.parser')
    base = []
    for conteneur_unique in conteneurs:
        try:
            conteneurs = soup.find_all('div', {'class': 'listings-cards__list-item'})
            details = conteneur_unique.find('div', {'class': 'listing-card__header__title'}).text.strip()
            etat = conteneur_unique.find('span', {'class': 'listing-card__header__tags__item listing-card__header__tags__item--condition listing-card__header__tags__item--condition_new'}).text
            adresse = conteneur_unique.find('div', {'class':'listing-card__header__location'}).text.strip().replace(',\n','').strip()
            prix = soup.find('span', {'class': 'listing-card__price__value 1'}).text.strip().replace('\u202f', '').replace(' F Cfa', '')
            image_lien = soup.find('img', {'class': 'listing-card__image__resource vh-img'})['src']

            dict =  {'details':details, 'etat':etat, 'adresse':adresse, 'prix':prix, 'image_lien':image_lien}
            base.append(dict)
        except:
             pass
    
    base_ = pd.DataFrame(base)
    base_donnee = pd.concat([base_, base_donnee], axis = 0).reset_index(drop = True)

In [135]:
# affichage de la base finale
base_donnee.head()

,details,etat,adresse,prix,image_lien
0,Réfrigérateur ASTECH Combiné 5 tiroirs 250litres,Neuf,Plateau ...,125900,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
1,CONGELATEUR HORIZONTAL - MIDEA HS-200 - 200L ...,Neuf,Ouakam ...,125900,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
2,Réfrigérateur SIDE BY SIDE LG,Neuf,Plateau ...,125900,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
3,REFRIGERATEUR ELECTROCOOL - 3 TIROIRS + FONTAI...,Neuf,Ouakam ...,125900,https://i.roamcdn.net/hz/ed/listing-thumb-224w...
4,REFRIGERTAEUR FINIX - MINI BAR - GRIS -65LITRE...,Neuf,Ouakam ...,125900,https://i.roamcdn.net/hz/ed/listing-thumb-224w...


In [136]:
# vérification de la dimension finale
base_donnee.shape

(2148, 5)

In [137]:
# enrégistrement de la base

base_donnee.to_csv('base_donnee_url1.csv', index = False, header = True)
base_donnee.to_excel('base_donnee_url1.xlsx', index = False, header = True)